<a href="https://colab.research.google.com/github/younghwanjo/2022-summer-LabIntern/blob/main/Deep%20Learning/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) prepare data
# 1) Design Model (input size, output size, forward pass)
# 2) Construct loss and optimizer
# 3) Training loop
#   - forward pass: compute prediction
#   - backward pass: compute gradients
#   - update weights

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 0) prepare data

batch_size = 100
num_classes = 10
num_epochs = 5
learning_rate = 0.01

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root="data",train=True,download=True,transform=transform)

test_dataset = torchvision.datasets.MNIST(root="data",train=False,download=True,transform=transform)

train_dataloader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

test_dataloader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)



In [ ]:
# 1) Design Model (input size, output size, forward pass)
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv2d(1,4,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(4,16,5)
        self.fc1 = nn.Linear(16*4*4, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = CNN()

In [ ]:
# 2) Construct loss and optimizer
learning_rate = 0.01
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
# 3) Training loop

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# images, labels = next(iter(train_dataloader))
# print(images.shape)
# conv1 = nn.Conv2d(1,4,5)
# pool = nn.MaxPool2d(2,2)
# conv2 = nn.Conv2d(4,16,5)
# x = conv1(images)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x = conv2(x)
# print(x.shape)
# x = pool(x)
# print(x.shape)

for t in range(num_epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.323333  [    0/60000]
loss: 2.315788  [10000/60000]
loss: 2.288524  [20000/60000]
loss: 2.285288  [30000/60000]
loss: 2.274367  [40000/60000]
loss: 2.264967  [50000/60000]
Test Error: 
 Accuracy: 53.8%, Avg loss: 2.195936 

Epoch 2
-------------------------------
loss: 2.188483  [    0/60000]
loss: 1.965721  [10000/60000]
loss: 1.564566  [20000/60000]
loss: 0.855220  [30000/60000]
loss: 0.742240  [40000/60000]
loss: 0.434076  [50000/60000]
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.423931 

Epoch 3
-------------------------------
loss: 0.414829  [    0/60000]
loss: 0.378835  [10000/60000]
loss: 0.462713  [20000/60000]
loss: 0.238466  [30000/60000]
loss: 0.218374  [40000/60000]
loss: 0.310396  [50000/60000]
Test Error: 
 Accuracy: 94.0%, Avg loss: 0.212466 

Epoch 4
-------------------------------
loss: 0.146667  [    0/60000]
loss: 0.183031  [10000/60000]
loss: 0.576748  [20000/60000]
loss: 0.140930  [30000/60000]
loss: 0.159334  [40000/6